Connecting runtime...

In [1]:
!nvidia-smi

Thu Mar  9 17:18:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Installing required libraries:

In [2]:
!pip install langchain
%pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.6/360.6 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 6.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.4 MB/s eta 0:00:00


In [3]:
! pip install -q langchain gpt-index llama-index transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 KB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [4]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor
import torch
from langchain.llms.base import LLM
from transformers import pipeline

In [5]:
class FlanLLM(LLM):
    model_name = "google/flan-t5-base"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]
 
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=FlanLLM())


In [6]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [7]:
text1 = """There’s no shortage of get-rich-quick schemes, from the latest crypto memecoin to flipping penny stocks. Don’t be fooled by their promises of easy wealth—schemes hide giant risks and the vast majority of investors end up losing money.

Instead, spend your time learning how to build wealth, which requires you to make an investing plan and adopt a long-term mindset. Follow these eight simple steps to get started building sustainable wealth.

1.  Start by Making a Plan
Building wealth starts with making a financial plan. That means taking the time to identify your goals and game out how you can accomplish them.

“Building wealth begins with a vision and a plan,” says Peter Cassciotta, owner of Asset Management and Advisory Services of Lee County.

Hiring a financial advisor is a great way to begin making your plan for building wealth. It’s a more expensive option, particularly for those who are just starting out, but choosing an advisor who’s a certified financial planner (CFP) means you’re paying for planning experience.

Shopping around for a robo-advisor that also offers access to financial advisors may be a more affordable option. Check out robos like Betterment or Ellevest, both of which provide managed investment portfolios plus the chance to talk with advisors.

Related: Find A Financial Advisor In 3 minutes

2.  Make a Budget and Stick to It
Plenty of people dread the “b” word, but budgeting is a key plank in your wealth building strategy. Building a budget and sticking to it helps increase your chances of carrying out your plan and achieving your financial goals.

Budgets also help you understand where your money goes each month and prevent behaviors that can endanger your goals, like overspending.

3.  Build Your Emergency Fund
When the furnace goes out or the refrigerator quits working, where does the money come from if you don’t have emergency savings? Lori Gross, financial and investment advisor at Outlook Financial Center says credit cards bear the brunt and cause you to incur extra costs and fees, like sky-high interest rates.

By building an emergency fund, you can protect your credit as well as reap the benefits of earning interest on an online savings account—all the while enjoying the peace of mind of knowing you have money in the bank to cover life’s surprises.

4.  Automate Your Financial Life
By making saving, investing and bill pay automatic, you all but eliminate the chance you forget to set aside money for your goals or make progress on paying off your debts.

That’s why Michael Morgan, president of TBS Retirement Planning, recommends that you have the aggregate amount you’ve budgeted for each of your expenses and goals automatically deducted from your paycheck and applied to each expense.

This is especially valuable when it comes to saving and investing, he says. “By doing so, you resist the temptation to spend rather than invest. Soon, you won’t miss the money that is being automatically deducted and your contributions will be made on a regular basis,” he says.
5.  Manage Your Debt
If you’re carrying a balance month to month, you aren’t alone: The average American has more than $90,000 in debt, according to Experian research.

Of course, not all debt is created equal—and some, like mortgages, may even be considered “good” debt, thanks to their general low interest rates and wealth building potential. Some experts even think of a mortgage payoff as a type of forced savings account because you’ll likely see at least a portion of your monthly payment back when you sell.

But if you’re rolling over a lot of bad debt, like high-interest credit card bills, every month, you may jeopardize your financial goals. That’s why it’s important to have a plan for your repayment, Gross says, with the ultimate goal of having a debt-free life.

If you aren’t sure how to get started, consider using the debt snowball or debt avalanche payoff methods. And remember: It’s possible (and often even advisable) to save money and pay down debt at the same time.

Then, as your balances fall, you’ll have even more money to put towards your emergency savings and investments.

6.  Max Out Your Retirement Savings
Uncle Sam gives you a few different ways to save up for retirement, and experts encourage you to take advantage of as many as you can. That means putting the most you can toward your employer’s retirement plan—think 401(k)—as well as individual retirement accounts (IRAs).

If contributing the legal maximum is going to be a stretch for you right now, make sure you’re at least saving enough to get any 401(k) match your company provides. That means if your employer offers a 3% match, you’re contributing at least 3% of your salary each pay period.

Don’t get discouraged if you can’t invest a lot to begin with. “Most of my clients invested a small amount of money for a long period of time,” says Casciotta. The power of compounding, then, helps turn these invested small sums into fortunes.

If you aren’t sure the best way to start investing within your 401(k) or IRA, consider a target-date fund or robo-advisor that manages a custom portfolio of funds based on the number of years you have until retirement.

7.  Stay Diversified
If you’re clinging to the idea that people only become wealthy by having highly concentrated positions—perhaps by holding large amounts of Bitcoin—consider loosening your grip. Having a diversified portfolio with different types of investments can both protect the wealth you’ve accumulated and position you to reap rewards even in market downturns.

“A diversified portfolio includes a mix of assets that do not necessarily move in the same direction and in the same magnitude at all times and is designed to help reduce volatility over time,” says Veronica Willis, investment strategy analyst at Wells Fargo Investment Institute.

8.  Up Your Earnings
While it isn’t a move that you can make at an online brokerage, investing in yourself by raising your income is an important step when it comes to how to build wealth. The more you earn over your lifetime, the more money you have available to invest.

“If you’ve been living comfortably on your current salary and you receive an increase, this is the perfect opportunity to begin the path to building wealth,” says Morgan, whether that means contributing more toward your retirement savings, paying down debt or bumping up your emergency fund savings,

In fact, financial expert Michael Kitces recommends you save at least half of every raise you get to position yourself for a secure retirement. This allows you to improve your quality of life gradually while also ensuring you don’t fall victim to standards of living that will be impossible for you to maintain in retirement.

If you don’t think you’re in the position to receive a raise, schedule time with your boss to determine what steps you need to take to advance in your current role. You may also consider picking up a side hustle or trying a passive income idea."""

In [8]:
#documents = SimpleDirectoryReader('data').load_data()

from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]

In [9]:
# set number of output tokens
num_output = 250
# set maximum input size
max_input_size = 512
# set maximum chunk overlap
max_chunk_overlap = 20


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [10]:
#index = GPTSimpleVectorIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

index = GPTListIndex(documents, embed_model=embed_model, llm_predictor=llm_predictor)

#index.save_to_disk('index.json')

In [11]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [13]:
response = index.query( "How do you build wealth in the long term?") 
print(response) 

Start by Making a Plan
